In [1]:
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.listdir('/content/drive/MyDrive/CIS4400_HateCrimes')

['hate_crime_raw.csv',
 'hate_crime_raw.gsheet',
 'hate_crimes_dictionary.gsheet']

In [19]:
filepath = '/content/drive/MyDrive/CIS4400_HateCrimes/hate_crime_raw.csv'
df = pd.read_csv(filepath)
df.head()

,incident_id,data_year,ori,pug_agency_name,pub_agency_unit,agency_type_name,state_abbr,state_name,division_name,region_name,population_group_code,population_group_description,incident_date,adult_victim_count,juvenile_victim_count,total_offender_count,adult_offender_count,juvenile_offender_count,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
0,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,3,"Cities from 50,000 thru 99,999",7/4/1991,NaN,NaN,1,NaN,NaN,Black or African American,Not Specified,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
1,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,3,"Cities from 50,000 thru 99,999",12/24/1991,NaN,NaN,1,NaN,NaN,Black or African American,Not Specified,2,Aggravated Assault;Destruction/Damage/Vandalism of Property,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
2,45,1991,AR0600300,North Little Rock,NaN,City,AR,Arkansas,West South Central,South,3,"Cities from 50,000 thru 99,999",7/10/1991,NaN,NaN,1,NaN,NaN,Black or African American,Not Specified,2,Aggravated Assault;Murder and Nonnegligent Manslaughter,2.0,Residence/Home,Anti-White,Individual,M,S
3,46,1991,AR0600300,North Little Rock,NaN,City,AR,Arkansas,West South Central,South,3,"Cities from 50,000 thru 99,999",10/6/1991,NaN,NaN,2,NaN,NaN,Black or African American,Not Specified,1,Intimidation,1.0,Residence/Home,Anti-White,Individual,S,S
4,47,1991,AR0670000,Sevier,NaN,County,AR,Arkansas,West South Central,South,8D,"Non-MSA counties under 10,000",10/14/1991,NaN,NaN,1,NaN,NaN,White,Not Specified,1,Intimidation,1.0,School/College,Anti-Black or African American,Individual,S,S


In [20]:
df.columns

Index(['incident_id', 'data_year', 'ori', 'pug_agency_name', 'pub_agency_unit',
       'agency_type_name', 'state_abbr', 'state_name', 'division_name',
       'region_name', 'population_group_code', 'population_group_description',
       'incident_date', 'adult_victim_count', 'juvenile_victim_count',
       'total_offender_count', 'adult_offender_count',
       'juvenile_offender_count', 'offender_race', 'offender_ethnicity',
       'victim_count', 'offense_name', 'total_individual_victims',
       'location_name', 'bias_desc', 'victim_types', 'multiple_offense',
       'multiple_bias'],
      dtype='object')

In [21]:
df.shape

(253776, 28)

In [22]:
# filter for data_year 2020 and later
df = df[df['data_year'] >= 2020].copy()

# convert your date column to YYYY-MM-DD
df.loc[:, 'incident_date'] = pd.to_datetime(df['incident_date']).dt.strftime('%Y-%m-%d')

# filter for only tristate data
df = df[df['state_name'].isin(['New York', 'New Jersey', 'Connecticut'])].copy()

# split the two columns on “/” into lists
df.loc[:, 'offense_name'] = df['offense_name'].str.split(r'/|;')
df.loc[:, 'location_name'] = df['location_name'].str.split(r'/|;')
df.loc[:, 'bias_desc'] = df['bias_desc'].str.split(';')

# explode both lists so each value gets its own row
df = df.explode('offense_name').explode('location_name').reset_index(drop=True)

# drop the specified columns
df = df.drop(columns=['pub_agency_unit', 'division_name', 'region_name', 'population_group_code'])

# drop NaN values
df = df.dropna()

# convert those columns to integers
float_cols = df.select_dtypes(include=['float']).columns
df[float_cols] = df[float_cols].astype(int)

# check row count
print(f"Rows after cleaning: {len(df)}")

Rows after cleaning: 8073


In [23]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


df.head(10)

,incident_id,data_year,ori,pug_agency_name,agency_type_name,state_abbr,state_name,population_group_description,incident_date,adult_victim_count,juvenile_victim_count,total_offender_count,adult_offender_count,juvenile_offender_count,offender_race,offender_ethnicity,victim_count,offense_name,total_individual_victims,location_name,bias_desc,victim_types,multiple_offense,multiple_bias
0,1327079,2020,CT0006400,Hartford,City,CT,Connecticut,"Cities from 100,000 thru 249,999",2020-05-28,0,1,1,0,1,Unknown,Unknown,1,Intimidation,1,School-Elementary,[Anti-Black or African American],Individual,S,S
1,1327079,2020,CT0006400,Hartford,City,CT,Connecticut,"Cities from 100,000 thru 249,999",2020-05-28,0,1,1,0,1,Unknown,Unknown,1,Intimidation,1,Secondary,[Anti-Black or African American],Individual,S,S
2,1327080,2020,CT0006400,Hartford,City,CT,Connecticut,"Cities from 100,000 thru 249,999",2020-02-20,1,0,1,1,0,White,Not Hispanic or Latino,1,Aggravated Assault,1,Highway,[Anti-Hispanic or Latino],Individual,S,S
3,1327080,2020,CT0006400,Hartford,City,CT,Connecticut,"Cities from 100,000 thru 249,999",2020-02-20,1,0,1,1,0,White,Not Hispanic or Latino,1,Aggravated Assault,1,Road,[Anti-Hispanic or Latino],Individual,S,S
4,1327080,2020,CT0006400,Hartford,City,CT,Connecticut,"Cities from 100,000 thru 249,999",2020-02-20,1,0,1,1,0,White,Not Hispanic or Latino,1,Aggravated Assault,1,Alley,[Anti-Hispanic or Latino],Individual,S,S
5,1327080,2020,CT0006400,Hartford,City,CT,Connecticut,"Cities from 100,000 thru 249,999",2020-02-20,1,0,1,1,0,White,Not Hispanic or Latino,1,Aggravated Assault,1,Street,[Anti-Hispanic or Latino],Individual,S,S
6,1327080,2020,CT0006400,Hartford,City,CT,Connecticut,"Cities from 100,000 thru 249,999",2020-02-20,1,0,1,1,0,White,Not Hispanic or Latino,1,Aggravated Assault,1,Sidewalk,[Anti-Hispanic or Latino],Individual,S,S
7,1400704,2020,CT0007700,Manchester,City,CT,Connecticut,"Cities from 50,000 thru 99,999",2020-06-21,0,3,2,2,0,White,Unknown,1,Intimidation,3,Highway,[Anti-Black or African American],Individual,S,S
8,1400704,2020,CT0007700,Manchester,City,CT,Connecticut,"Cities from 50,000 thru 99,999",2020-06-21,0,3,2,2,0,White,Unknown,1,Intimidation,3,Road,[Anti-Black or African American],Individual,S,S
9,1400704,2020,CT0007700,Manchester,City,CT,Connecticut,"Cities from 50,000 thru 99,999",2020-06-21,0,3,2,2,0,White,Unknown,1,Intimidation,3,Alley,[Anti-Black or African American],Individual,S,S


In [24]:
print(len(df))

8073


In [25]:
df.isnull().sum()

,0
incident_id,0
data_year,0
ori,0
pug_agency_name,0
agency_type_name,0
state_abbr,0
state_name,0
population_group_description,0
incident_date,0
adult_victim_count,0


In [26]:
df.to_csv('hate_crime_cleaned.csv', index = False)
print('Complete')

Complete
